# Embedding Translations


# 01 Setup


In [6]:
### LOCAL INSTALLS ###
%pip install torch
%pip install torchvision
%pip install torchaudio
%pip install datasets

  Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl (63.9 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl (12 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached torchvision-0.20.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached pillow-11.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
Using cached torchvision-0.20.1-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)
Using cached pillow-11.0.0-cp312-cp312-macosx_1

In [10]:
### GENERAL IMPORTS ###

### ML IMPORTS ###

import torch
from datasets import load_dataset

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


device = torch.device("cuda:0")
# assert torch.cuda.is_available()

PyTorch version: 2.5.1
CUDA available: False


## 02 Data


Data source ideas:

- ?


In [12]:
dataset = load_dataset("tweet_eval", "emotion", split="train[:1000]")  # Let's start with just 1000 examples

In [14]:
dataset.to_polars()

text,label
str,i64
"""“Worry is a down payment on a …",2
"""My roommate: it's okay that we…",0
"""No but that's so cute. Atsu wa…",1
"""Rooneys fucking untouchable is…",0
"""it's pretty depressing when u …",3
…,…
"""@user DirectTV is the best. Ap…",0
"""Ok but how are the fast &amp; …",1
"""@user #terrible Paul so deserv…",1


## 03 Model


## 04 Training


## 05 Visualization/Results

I recommend we use plot.ly (MantisAI is also something we can use)
